In [6]:
import numpy as np
import random as rnd

In [7]:
import igl
import scipy as sp
import meshplot as mp
import numpy as np
from meshplot import plot, subplot, interact
import math

In [8]:
v, f = igl.read_triangle_mesh("3dPotteryDs/3D Models/Lagynos/qpvase5.obj")

In [9]:
f.shape, type(f), v.shape, type(v)

((1960, 3), numpy.ndarray, (1024, 3), numpy.ndarray)

In [10]:
def correctDirection(nVector):
    nVector = nVector * np.sign(nVector[np.argmax(abs(nVector))])
    return nVector

In [11]:
TH_RATIO_CRV = 0.9
TH_COSINE = 0.75
#TH_MN_CURVATURE = 0.025

In [12]:
#Compute facet normals
n_faces = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
#Compute vertex normals
n_vertex = igl.per_vertex_normals(v, f)
#v1, v2: principal directions | k1, k2: principal values
v1, v2, k1, k2 = igl.principal_curvature(v, f, 2, True)

In [18]:
v1, v2, k1, k2 = igl.principal_curvature(v, f)
k1[1], k2[1]

(0.07313080383835166, 6.049946462437392)

In [33]:
area_t = 0.5 * sum(igl.doublearea(v, f))
area_t

53.202287007756645

In [6]:
alst = igl.adjacency_list(f)
len(alst), alst[0]

(1024, [5, 11, 12, 13, 23])

In [15]:
def calculate_principal(ivx):
    alst = igl.adjacency_list(f)
    VF,VFi = igl.vertex_triangle_adjacency(f, len(v))
    ltf = []
    lvt = []
    lvt.append(ivx)
    #print("*", ivx, alst[ivx])

    for fx in VF[VFi[ivx]:VFi[ivx+1]]:
        #print(fx, f[fx])
        ltf.append(fx)

    for vr1 in alst[ivx]:
        if vr1 not in lvt:
            lvt.append(vr1)

        #print("*", vr1, alst[vr1])

        for fx in VF[VFi[vr1]:VFi[vr1+1]]:
            #print(fx, f[fx])
            if fx not in ltf:
                ltf.append(fx)

        for vr2 in alst[vr1]:
            if vr2 not in lvt:
                lvt.append(vr2)
                
    vx_dtnr = {}
    for ix, vtx in enumerate(lvt):
        vx_dtnr[vtx] = ix

    vc = np.array([v[lvt[i]] for i in range(len(lvt))])
    fc = np.array([[vx_dtnr[f[ltf[i]][0]], vx_dtnr[f[ltf[i]][1]], vx_dtnr[f[ltf[i]][2]]] for i in range(len(ltf))], dtype=int)
    v1c, v2c, k1c, k2c = igl.principal_curvature(vc, fc)
    return k1c[0], k2c[0]

In [16]:
calculate_principal(1)

(-0.09154242832482255, 4.914515534324624)

In [27]:
#Calculate curvatures for sampled points
length_x = max(v[:][0]) - min(v[:][0])
length_y = max(v[:][1]) - min(v[:][1])
length_z = max(v[:][2]) - min(v[:][2])

area = 2 * (length_x * length_y + length_x * length_z + length_y * length_z)
diagonal_bbox = math.sqrt(length_x * length_x + length_y * length_y + length_z * length_z)
v_size = min(round(area * 125), round(v.shape[0] / 2))

v_ratio = round(v.shape[0] / v_size)
#pk1 = np.percentile(abs(k1), 10)
#pk2 = np.percentile(abs(k2), 10)
v_indices = [] #np.array([0 for i in range(v_size)])
k1_c = []
k2_c = []

from datetime import datetime
start = datetime.now()

for i in range(0, v_size):
    index = rnd.randint(v_ratio * i, min(v_ratio * (i + 1) - 1, v.shape[0] - 1))
    #if abs(k1[index]) > pk1 or abs(k2[index]) > pk2:
    v_indices.append(index) #v_indices[i] = index;
    k1c, k2c = calculate_principal(index)
    k1_c.append(k1c)
    k2_c.append(k2c)

end = datetime.now()
print(end - start)

0:00:01.526064


In [32]:
for i in range(0, len(v_indices)):
    print(v_indices[i], k1[v_indices[i]], k1_c[i], k2[v_indices[i]], k2_c[i], k[v_indices[i]])

1 0.07313080383835166 -0.09154242832482255 6.049946462437392 4.914515534324624 3.141592653589793
3 0.06204131416569056 -0.11937004836433021 7.0031067580404 6.037003979950763 3.141592653589793
4 0.04984427297711365 -0.26616511487362604 5.402516809055341 5.09323660862367 3.141592653589793
7 0.04914707629371536 -0.37966347326509076 6.675502369946603 5.551859047657458 3.1415926535897936
9 0.06119471350205668 -0.11002425831081084 5.702190446114511 4.900939667145079 3.1415926535897927
11 0.11396330071585647 -0.11703512876994691 7.371292887097482 5.98500496605954 3.1415926535897927
12 0.07329948414062525 0.046936483725542995 5.308657157797163 4.409818247617244 -0.037937248004687896
15 0.08947648543259511 0.011592246002053986 5.275059058765836 4.345962452224432 -0.03217514316392778
17 0.003741167169015139 0.043407633198932975 5.395809313646195 5.134518605521143 -0.041707858939710296
18 0.03539212444329404 0.04577901240902757 5.48296980427355 5.116949221850113 -0.041468880325711055
20 0.0997333

203 -0.00022551792973192006 0.6333448806416339 0.7617841223686681 10.868878978036868 3.0348672495070836
204 -0.018072305390461646 0.6332786453306878 0.8541517639268198 10.868397190803714 3.034913279775533
206 0.019809665881139818 0.633264023736329 0.8319907555948697 10.868384028742241 3.0348988536575088
209 0.03888638685658758 0.7783012444145067 0.8816886822174099 9.193002820923315 3.034892899626966
211 -0.0771911467263197 0.6370404027588147 0.9849522391373102 11.11387239899989 3.0348672505344467
213 0.0676829191076825 0.6333229108103394 0.9045049645477367 10.868578000601243 3.0348752791110134
214 0.12434422177279258 0.7783207439081858 0.8402842073849693 9.19354166285973 3.0348854935745426
216 0.1040243402720465 0.5977501747737358 0.7963012811313285 11.975711919280862 3.0348527000434617
218 0.11181847916423426 0.6332279422982185 0.816353760506294 10.868469971394896 3.034898854565188
220 0.08720885327798268 0.6332786453306878 0.8828923638112767 10.868397190803714 3.034913279775533
223 0

583 -0.9039853472449215 -0.018959224970019794 0.4182017019720664 0.45280379666455145 0.022798543947665784
584 -0.9203773280353761 -0.037055266412322825 0.39843948741494045 0.4653886128484253 0.017692448157265783
587 -0.8972463838517404 -0.17026016732314317 0.3584120344967988 0.4464785668132574 0.007848895678997048
589 -0.9007469027318671 -0.18939416697011324 0.3131761726512203 0.4115542010330646 0.007848215819974635
590 -0.8995772654964298 -0.15240398841484418 0.34974127546022876 0.36685703627092536 0.009986225532478743
592 -0.917794929398151 -0.03798244515214792 0.36806473906516823 0.4663516491192105 0.017697321115689446
595 -0.8681485402422936 0.027524581729528153 0.37795058744650734 0.549418349534641 0.03453207944787451
596 -0.8700383634483694 0.04742419481498506 0.3541686827970859 0.44412386489210126 0.04091544817260506
599 -0.9107540789210029 0.17145752095017958 0.4023561011859317 0.6005777216115664 0.06046631641393119
601 -0.9082369699860192 0.2054218841837468 0.4215216791394595 

749 -0.5361503228973081 -0.062063289220862634 1.482324081141186 2.874176279787751 0.18281763273659601
751 -0.48046896234258846 -0.056028986211986764 1.6996363906830216 2.7546317523655994 0.18271533672234086
753 -0.5328131847496695 -0.04167373625001699 1.6308775442103929 3.0087566502370313 0.1828563087809567
755 -0.6433935510546912 -0.05236015438659656 1.098823063049288 2.962770285300018 0.182806312705315
757 -0.504067508460451 -0.07774813784139792 1.1897356674373543 2.622415061904442 0.18277103138259743
758 -0.5037001878872046 -0.07773186909272513 1.5284840967858384 2.6224315600891157 0.18279798484455068
760 -0.4908873214074457 -0.07772117777623971 1.3582629819244536 2.6223719754086443 0.18279439133549324
762 -0.4963512167676071 -0.07776622919712245 1.3812085208299507 2.6221476311417957 0.18273356270841346
764 -0.5002790417369268 -0.07771415469990846 1.4103692179002805 2.6224122092478637 0.1828276031427798
766 -0.489088626246524 -0.07773558678276728 1.6239771825282088 2.622205940296244

In [31]:
k2[1017], k2[1019], k2[1020], k2[1023]

(0.0011272983279000993,
 0.001416974016115581,
 0.0014163440468872037,
 0.00010758960198952385)

In [7]:
VF,VFi = igl.vertex_triangle_adjacency(f, len(v))
ltf = []
lvt = []

ivx = 0
lvt.append(ivx)
print("*", ivx, alst[ivx])

for fx in VF[VFi[ivx]:VFi[ivx+1]]:
    print(fx, f[fx])
    ltf.append(fx)
            
for vr1 in alst[ivx]:
    if vr1 not in lvt:
        lvt.append(vr1)
        
    print("*", vr1, alst[vr1])
    
    for fx in VF[VFi[vr1]:VFi[vr1+1]]:
        print(fx, f[fx])
        if fx not in ltf:
            ltf.append(fx)
    
    for vr2 in alst[vr1]:
        if vr2 not in lvt:
            lvt.append(vr2)
        
print(sorted(ltf), len(ltf))
print(lvt, len(lvt)) #sorted

* 0 [5, 11, 12, 13, 23]
1936 [ 0 11 23]
1937 [ 0 23 13]
1958 [12  5  0]
1959 [13 12  0]
* 5 [0, 9, 12]
1946 [ 9  5 12]
1958 [12  5  0]
* 11 [0, 6, 14, 23]
1936 [ 0 11 23]
1938 [11  6 14]
1939 [11 14 23]
* 12 [0, 5, 9, 13, 21, 24, 33]
1922 [21 12 33]
1923 [12 24 33]
1934 [24 12 13]
1946 [ 9  5 12]
1947 [ 9 12 21]
1958 [12  5  0]
1959 [13 12  0]
* 13 [0, 12, 23, 24, 25, 35]
1912 [13 23 35]
1913 [13 35 25]
1934 [24 12 13]
1935 [25 24 13]
1937 [ 0 23 13]
1959 [13 12  0]
* 23 [0, 11, 13, 14, 35]
1912 [13 23 35]
1914 [23 14 35]
1936 [ 0 11 23]
1937 [ 0 23 13]
1939 [11 14 23]
[1912, 1913, 1914, 1922, 1923, 1934, 1935, 1936, 1937, 1938, 1939, 1946, 1947, 1958, 1959] 15
[0, 5, 9, 12, 11, 6, 14, 23, 13, 21, 24, 33, 25, 35] 14


In [8]:
vx_dtnr = {}

for ix, vtx in enumerate(lvt):
    vx_dtnr[vtx] = ix
    
vx_dtnr

{0: 0,
 5: 1,
 9: 2,
 12: 3,
 11: 4,
 6: 5,
 14: 6,
 23: 7,
 13: 8,
 21: 9,
 24: 10,
 33: 11,
 25: 12,
 35: 13}

In [9]:
vc = np.array([v[lvt[i]] for i in range(len(lvt))])
vc

array([[ 1.68042e+00,  1.93480e-01,  5.33990e+00],
       [ 1.52410e+00,  2.49790e-01,  5.33990e+00],
       [ 1.39859e+00,  2.20140e-01,  5.33990e+00],
       [ 1.52410e+00,  2.49790e-01,  5.14476e+00],
       [ 1.73619e+00,  1.07320e-01,  5.33990e+00],
       [ 1.76305e+00, -1.63000e-03,  5.33990e+00],
       [ 1.76305e+00, -1.63000e-03,  5.14476e+00],
       [ 1.73619e+00,  1.07320e-01,  5.14476e+00],
       [ 1.68042e+00,  1.93480e-01,  5.14476e+00],
       [ 1.39859e+00,  2.20140e-01,  5.14476e+00],
       [ 1.52506e+00,  2.72640e-01,  4.80208e+00],
       [ 1.38808e+00,  2.40280e-01,  4.80208e+00],
       [ 1.69568e+00,  2.11170e-01,  4.80208e+00],
       [ 1.75655e+00,  1.17140e-01,  4.80208e+00]])

In [10]:
fc = np.array([[vx_dtnr[f[ltf[i]][0]], vx_dtnr[f[ltf[i]][1]], vx_dtnr[f[ltf[i]][2]]] for i in range(len(ltf))], dtype=int)
fc

array([[ 0,  4,  7],
       [ 0,  7,  8],
       [ 3,  1,  0],
       [ 8,  3,  0],
       [ 2,  1,  3],
       [ 4,  5,  6],
       [ 4,  6,  7],
       [ 9,  3, 11],
       [ 3, 10, 11],
       [10,  3,  8],
       [ 2,  3,  9],
       [ 8,  7, 13],
       [ 8, 13, 12],
       [12, 10,  8],
       [ 7,  6, 13]])

In [11]:
#v1, v2: principal directions | k1, k2: principal values
v1c, v2c, k1c, k2c = igl.principal_curvature(vc, fc)
k1c[0], k2c[0]

(-0.28269719783907926, 5.3767229408838935)

In [4]:
k = igl.gaussian_curvature(v, f)
k[0]

3.141592653589793

In [34]:
type(k1)

numpy.ndarray

In [12]:
k1[0], k2[0]

(0.08726475567419531, 5.672147490389231)

In [15]:
k1[0], k2[0]

(-0.2826971978390795, 5.376722940883893)

In [31]:
length_x = max(v[:][0]) - min(v[:][0])
length_y = max(v[:][1]) - min(v[:][1])
length_z = max(v[:][2]) - min(v[:][2])

area = 2 * (length_x * length_y + length_x * length_z + length_y * length_z)
diagonal_bbox = math.sqrt(length_x * length_x + length_y * length_y + length_z * length_z)

In [32]:
v_size = min(round(area * 125), round(v.shape[0] / 2))

In [56]:
v_size, round(v.shape[0] / v_size)

(512, 2)

In [267]:
np.percentile(abs(k1), 15), np.percentile(abs(k1), 40), np.percentile(abs(k1), 65), np.percentile(abs(k1), 90)

(0.10252667281446885,
 0.3194790599674767,
 0.5525401716109815,
 1.7620566621015445)

In [268]:
np.percentile(abs(k2), 15), np.percentile(abs(k2), 40), np.percentile(abs(k2), 65), np.percentile(abs(k2), 90)

(0.26357892002127953,
 0.5451515355199719,
 1.6161543821360465,
 5.097082818947928)

In [57]:
v_ratio = round(v.shape[0] / v_size)
pk1 = np.percentile(abs(k1), 10)
pk2 = np.percentile(abs(k2), 10)
v_indices = [] #np.array([0 for i in range(v_size)])

for i in range(0, v_size):
    index = rnd.randint(v_ratio * i, min(v_ratio * (i + 1) - 1, v.shape[0] - 1))
    if abs(k1[index]) > pk1 or abs(k2[index]) > pk2:
        v_indices.append(index) #v_indices[i] = index;

In [59]:
s_size = min(round(area * 25), round(len(v_indices)/2))
s_indices = np.array([0 for i in range(s_size)])
s_size, v_size, s_indices.shape, len(v_indices), v_ratio, v.shape[0]

(254, 512, (254,), 509, 2, 1024)

In [60]:
s_ratio = round(len(v_indices) / s_size)

for i in range(0, s_size):
    index = rnd.randint(s_ratio * i, min(s_ratio * (i + 1) - 1, len(v_indices) - 1));
    s_indices[i] = v_indices[index];

In [482]:
# iPoints : indexes list of points Sampled
def baseReflection(iPoints): 
    sample_size = len(iPoints)
    lxPairs = []
    
    for i in range(0, sample_size - 1):
        for j in range(i + 1, sample_size):
            ix = iPoints[i]
            jx = iPoints[j]
            
            ratio_1 = abs(k1[ix] / k1[jx])
            ratio_2 = abs(k2[ix] / k2[jx])
            
            if TH_RATIO_CRV < ratio_1 < 1/TH_RATIO_CRV and TH_RATIO_CRV < ratio_2 < 1/TH_RATIO_CRV:
                reflexNormal = v[ix] - v[jx]
                reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)
                    
                normalBalance = n_vertex[ix] + n_vertex[jx]
                normalBalance = normalBalance / np.linalg.norm(normalBalance)
                                
                if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
                    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
                        reflexNormal = -reflexNormal
                    
                    middlePoint = (v[ix] + v[jx]) / 2                    
                    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
                    origin = np.array([0, 0, 0])
                    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D) 
                    
                    #Use projection point instead of D-component
                    xcPair = [ix, jx, reflexNormal, projection]
                    lxPairs.append(xcPair)
                   
    return lxPairs

In [273]:
lxcPairs = baseReflection(s_indices)
len(lxcPairs)

512

In [404]:
lxcPairs[22]

[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])]

In [483]:
for i in range(0, len(lxcPairs)):
    distanceMetric = distanceMetricRx(lxcPairs[22], lxcPairs[i])
    if distanceMetric[0] < 0.01 and distanceMetric[1] < 0.25: #relative distance, angle
        print(i, distanceMetric)

22 (0.0, 0.0)
27 (1.0577484483319858e-07, 0.09817492626985724)
78 (2.3141119828297346e-08, 0.09817454387963194)
185 (0.002453352188755538, 0.19634849685190398)
186 (0.003413188580357813, 0.09817467710707292)
227 (0.0024530076321578613, 0.19634908590380687)
233 (0.0028038893131595703, 0.09816014393018387)
241 (0.003672185893630966, 0.19635707823462706)
249 (0.0024530670812220505, 0.19634968064141764)
250 (0.003121760987498162, 0.0)
254 (0.0036715374832441507, 0.19635198782794855)
265 (1.481113423227101e-07, 0.19635231157050412)
269 (3.795926127280127e-07, 2.7739383084482916e-06)
271 (0.0, 0.0)
278 (0.005041819926094839, 0.09817225399172687)
281 (0.006143674411385388, 0.09817806913206695)
295 (0.005041376926852472, 0.09816941695128027)
302 (0.0050405830447189616, 0.09815470335736799)
363 (0.004415535600215444, 0.19635226916952062)
364 (0.005620062736515237, 7.818779996154839e-06)
368 (0.006608250494281115, 0.19635008725626182)
371 (0.005041861329267942, 0.09817150176618136)
374 (0.006607

In [201]:
len(lxcPairs), type(lxcPairs[1]), type(lxcPairs[1][2]), type(lxcPairs[1][2][2])

(512, list, numpy.ndarray, numpy.float64)

In [203]:
ax = 134
bx = 183
v[ax] - v[bx], n_vertex[ax] + n_vertex[bx], np.dot(v[ax] - v[bx], n_vertex[ax] + n_vertex[bx]), k1[ax], k2[ax], k1[bx], k2[bx]

(array([ 0.73575, -0.01815, -0.12484]),
 array([-0.25846642, -1.36020535, -1.40388536]),
 0.00978210572280025,
 0.5158379675871669,
 5.419074537774634,
 0.5185426111136144,
 5.508625957435771)

In [67]:
def projectPointPlane(point, plane, planeD):
    k = point.dot(plane) + planeD
    return point - k * plane

In [561]:
def distanceMetricRx(rxPairA, rxPairB):
    module = np.linalg.norm(rxPairA[2]) * np.linalg.norm(rxPairA[2])
    angle = math.acos(min(1.0, abs(np.dot(rxPairA[2], rxPairB[2])) / module))
    
    origin = np.array([0, 0, 0])
    #projectionA = projectPointPlane(origin, rxPairA[2], rxPairA[3])
    #projectionB = projectPointPlane(origin, rxPairB[2], rxPairB[3])
    distance = np.linalg.norm(rxPairA[3] - rxPairB[3])
    
    return distance / diagonal_bbox, angle

In [184]:
diagonal_bbox

9.314047731018992

In [258]:
def epanechnikov_weight(distance, kernel_bandwidth):
    c_distance = distance[0] * distance[1]
    c_kernel_bandwidth = kernel_bandwidth[0] * kernel_bandwidth[1]
    return max(0.0, 1.0 - (c_distance / c_kernel_bandwidth)) 
    #c_kernel_bandwidth **2

In [522]:
def shiftPair_Rx(rxPair, rxList, kernel_bandwidth):
    totalW = 0.0
    shifted_pair = [rxPair[0], rxPair[1], np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0])]
    
    for i in range(len(rxList)):
        distanceMetric = distanceMetricRx(rxPair, rxList[i])
        
        if distanceMetric[0] < kernel_bandwidth[0] and distanceMetric[1] < kernel_bandwidth[1]:           
            # A list cannot directly handle a mathematical operations, while array can
            weight = epanechnikov_weight(distanceMetric, kernel_bandwidth)
            shifted_pair[2] += rxList[i][2] * weight
            shifted_pair[3] += rxList[i][3] * weight
            totalW += weight
    
    if totalW > 0:
        shifted_pair[2] = shifted_pair[2] / np.linalg.norm(shifted_pair[2])
        shifted_pair[3] = shifted_pair[3] / totalW
        
        return shifted_pair
    else:
        return rxPair

In [278]:
EPSILON = 0.05

#distance / bounding_box, angle
KERNEL_BW = [0.05, 0.05]

In [231]:
KERNEL_BW[0], KERNEL_BW[1]

(0.001, 0.1)

In [289]:
#lxcPairs
def meanShift_Rx(lxPairs, kernel_bandwidth):
    stop_moving = [False for i in range(len(lxPairs))]
    shiftedPairs = lxPairs.copy()
        
    while True:
        max_shift_distance = 0
        
        for i in range(len(lxPairs)):
            if not stop_moving[i]:
                #print('x = ', i, shiftedPairs[i])
                pair_new = shiftPair_Rx(shiftedPairs[i], lxPairs, kernel_bandwidth)  
                
                #print('y = ', i, pair_new)
                distance_metric = distanceMetricRx(pair_new, shiftedPairs[i])
                shift_distance = distance_metric[0] * distance_metric[1]
                max_shift_distance = max(max_shift_distance, shift_distance)
                shiftedPairs[i] = pair_new
                
                if shift_distance < EPSILON:
                    stop_moving[i] = True
                
        if max_shift_distance < EPSILON:
            break
            
    return shiftedPairs

In [292]:
shiftedPairsFx = meanShift_Rx(lxcPairs, KERNEL_BW)

[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 22 [259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] (0.0, 0.0)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 250 [550, 570, array([ 0.70710678,  0.70710678, -0.        ]), array([-0.02056, -0.02056,  0.     ])] (0.003121760987498162, 0.0)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 269 [584, 600, array([ 0.70710874,  0.70710482, -0.        ]), array([2.50001387e-06, 2.50000000e-06, 0.00000000e+00])] (3.795926127280127e-07, 2.7739383084482916e-06)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 271 [587, 597, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] (0.0, 0.0)
[259, 285, array([ 0.70710678,  0.70710678, -0.        ]), array([0., 0., 0.])] 364 [773, 795, array([ 0.70710125,  0.70711231, -0.        ]), array([-0.03701359, -0.03701417,  0.        ])] (0

In [312]:
len(shiftedPairsFx)

512

In [486]:
#distance / bounding_box, angle
CLUSTER_EP = [0.1, 0.1]

#distances_line / #distance_projections
CLUSTER_RT = [0.05, 0.05]

In [371]:
def clusterRx(pairs, shiftedPairs):
    cluster_set = []
    
    for i in range(0, len(shiftedPairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRx(shiftedPairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points, shifted points
            cluster = [shiftedPairs[i], [], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
        cluster_set[c][3].append(shiftedPairs[i])
    
    return cluster_set

In [391]:
n_cluster = clusterRx(lxcPairs, shiftedPairsFx)
#n_cluster

In [392]:
len(n_cluster)

128

In [528]:
n_cluster[9]

[[127,
  159,
  array([ 0.83811743, -0.32647235,  0.43700684]),
  array([ 1.88460393, -0.73419254,  0.98287473])],
 [11, 54, 58, 68, 71, 123, 154],
 [[127,
   159,
   array([ 0.8375182 , -0.32672394,  0.43796659]),
   array([ 2.01971781, -0.78791142,  1.05617875])],
  [291,
   363,
   array([ 0.87575126, -0.31365869,  0.36698496]),
   array([ 1.54508682, -0.55338762,  0.6474711 ])],
  [291,
   395,
   array([ 0.86620311, -0.29372365,  0.40425065]),
   array([ 1.67066883, -0.56651257,  0.7796889 ])],
  [292,
   363,
   array([ 0.84831528, -0.34970207,  0.39757974]),
   array([ 1.6214583 , -0.66841579,  0.75992852])],
  [292,
   395,
   array([ 0.83873628, -0.32621157,  0.43601315]),
   array([ 1.74479847, -0.67860837,  0.90702537])],
  [313,
   401,
   array([ 0.82535571, -0.3938982 ,  0.40451471]),
   array([ 1.60413969, -0.7655702 ,  0.78620417])],
  [342,
   371,
   array([ 0.83053951, -0.37785531,  0.40918149]),
   array([ 1.57229828, -0.71531968,  0.77462343])]],
 [[127,
   159,
  

In [419]:
for x in range(0, len(n_cluster)):
    if len(n_cluster[x][1]) > 3:
        print(x, len(n_cluster[x][1]))

9 7
10 13
12 23
14 30
15 12
16 34
17 25
18 27
19 4
20 24
25 26
27 5
28 4
29 26
30 10
32 15
42 21
96 27
103 9
104 6


In [398]:
def clusterRx2(pairs):
    cluster_set = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRx(pairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [pairs[i], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
    
    return cluster_set

In [399]:
n_cluster2 = clusterRx2(lxcPairs)

In [418]:
for x in range(0, len(n_cluster2)):
    if (22 in n_cluster2[x][1]):
        print(x)

18


In [420]:
for x in range(0, len(n_cluster2)):
    if len(n_cluster2[x][1]) > 3:
        print(x, len(n_cluster2[x][1]))

9 7
10 12
12 23
14 30
15 13
16 34
17 24
18 27
20 24
25 26
27 4
28 4
29 26
30 10
32 20
43 21
78 4
93 28
101 9


In [427]:
def normalized(X):
    return X / np.linalg.norm(X)

In [428]:
A = [1/3, 1/3, 1/3, -3]
B = [1, 0, 0, -2]

U = normalized(np.cross(A[:-1], B[:-1]))
M = np.array((A[:-1], B[:-1], U))
X = np.array((-A[-1], -B[-1], 0.))

In [432]:
U, M, X, np.linalg.solve(M, X)

(array([ 0.        ,  0.70710678, -0.70710678]),
 array([[ 0.33333333,  0.33333333,  0.33333333],
        [ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.70710678, -0.70710678]]),
 array([3., 2., 0.]),
 array([2. , 3.5, 3.5]))

In [456]:
def projectPointLine(point, line, pointLine):
    ap = point - pointLine
    ab = line
    return pointLine + np.dot(ap, ab)/np.dot(ab, ab) * ab

In [477]:
def distancePointLine(point, line, pointLine):
    return np.linalg.norm(point - projectPointLine(point, line, pointLine))

In [479]:
distancePointLine(np.array([0, 0, 0]), np.array([0.31361367, 0.27279294, 0.90952212]), np.array([1, 2, 3]))

1.0620432616075952

In [460]:
projectPointLine(np.array([0, 0, 0]), np.array([0.31361367, 0.27279294, 0.90952212]), np.array([1, 2, 3]))

array([-0.12517242,  1.0212828 , -0.26315243])

In [461]:
np.dot([-0.12517242,  1.0212828 , -0.26315243], [0.31361367, 0.27279294, 0.90952212])

-4.523009744517026e-10

In [435]:
lxcPairs[1][2], lxcPairs[1][3], lxcPairs[2][2], lxcPairs[2][3]

(array([0.31361367, 0.27279294, 0.90952212]),
 array([1.56130636, 1.35808284, 4.52799987]),
 array([0.10147684, 0.0345258 , 0.99423861]),
 array([0.49051945, 0.16689105, 4.8059574 ]))

In [439]:
normalized(np.cross(lxcPairs[1][2], lxcPairs[2][2]))

array([ 0.73665951, -0.67427918, -0.05177213])

In [466]:
-np.dot(lxcPairs[1][2], lxcPairs[1][3]), -np.dot(lxcPairs[2][2], lxcPairs[2][3])

(-4.978438444982922, -4.833806832055721)

In [480]:
def planesIntersection(pairM, pairN):
    U = normalized(np.cross(pairM[2], pairN[2]))
    M = np.array((pairM[2], pairN[2], U))
    X = np.array((np.dot(pairM[2], pairM[3]), np.dot(pairN[2], pairN[3]), 0.))
    
    origin = np.array([0, 0, 0])
    return correctDirection(U), projectPointLine(origin, U, np.linalg.solve(M, X))

In [481]:
L = planesIntersection(lxcPairs[1], lxcPairs[2])
L

(array([ 0.73665951, -0.67427918, -0.05177213]),
 array([1.32836568, 1.09128052, 4.6883425 ]))

In [467]:
pM = lxcPairs[1]
pN = lxcPairs[2]
np.dot(L[1],pM[2]) - np.dot(pM[2],pM[3]), np.dot(L[1],pN[2]) - np.dot(pN[2],pN[3])

(0.0, -8.881784197001252e-16)

In [470]:
k1[lxcPairs[1][0]], k2[lxcPairs[1][0]]

(0.08947648543259511, 5.275059058765836)

In [493]:
not_parallel = np.dot(lxcPairs[2][2], lxcPairs[3][2]) < TH_RATIO_CRV
not_parallel, np.dot(lxcPairs[2][2], lxcPairs[3][2]), lxcPairs[2][2], lxcPairs[3][2]

(False,
 0.9986857980835615,
 array([0.10147684, 0.0345258 , 0.99423861]),
 array([ 0.13308172, -0.00571861,  0.99108857]))

In [505]:
def BaseContinuousRotation(pairs):
    lrPairs = []
    for m in range(0, len(pairs) - 1):
        for n in range(m+1, len(pairs)):
            rpM = pairs[m]
            rpN = pairs[n]            
            ratio_k1 = max(k1[rpM[0]], k1[rpM[1]]) / max(k1[rpN[0]], k1[rpN[0]])
            ratio_k2 = max(k2[rpM[0]], k2[rpM[1]]) / max(k2[rpN[0]], k2[rpN[0]])
            not_parallel = np.dot(rpM[2], rpN[2]) < TH_RATIO_CRV
            
            if TH_RATIO_CRV < ratio_k1 < 1 / TH_RATIO_CRV and TH_RATIO_CRV < ratio_k2 < 1 / TH_RATIO_CRV and not_parallel:
                #print(m, rpM[2], n, rpN[2])
                line = planesIntersection(rpM, rpN)
                
                p1_i = v[rpM[0]]
                p2_i = v[rpN[0]]                
                distance_1i = distancePointLine(p1_i, line[0], line[1])
                projection_1i = projectPointLine(p1_i, line[0], line[1])
                
                distance_2i = distancePointLine(p2_i, line[0], line[1])
                projection_2i = projectPointLine(p2_i, line[0], line[1])
                
                if np.linalg.norm(projection_1i - projection_2i) < CLUSTER_EP[0] and distance_1i - distance_2i < CLUSTER_EP[0]:
                    #line[0], line[1]
                    rtPair = [rpM[0], rpM[1], rpN[0], rpN[1], line]
                    lrPairs.append(rtPair)
    return lrPairs

In [506]:
lcrPairs = BaseContinuousRotation(lxcPairs)

In [507]:
len(lcrPairs)

5057

In [540]:
0.5*lcrPairs[4][4][0], 0.5*lcrPairs[4][4][1]

(array([-0. ,  0. ,  0.5]),
 array([3.03754315e-06, 1.68672731e-06, 0.00000000e+00]))

In [545]:
shifted_pair = [lcrPairs[4][0], lcrPairs[4][1], lcrPairs[4][2], lcrPairs[4][3], [np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0])]]

In [546]:
shifted_pair

[259, 260, 259, 273, [array([0., 0., 0.]), array([0., 0., 0.])]]

In [518]:
va = np.array([-0.,  0.,  1.])
vb = np.array([-0.,  0.,  1.])
vc = np.dot(va, vb)
math.acos(vc)

0.0

In [562]:
def distanceMetricRt(rtPairA, rtPairB):
    module = np.linalg.norm(rtPairA[4][0]) * np.linalg.norm(rtPairB[4][0])
    angle = math.acos(min(1.0, abs(np.dot(rtPairA[4][0], rtPairB[4][0])) / module))
        
    origin = np.array([0, 0, 0])
    distance = np.linalg.norm(rtPairA[4][1] - rtPairB[4][1])
    
    return distance / diagonal_bbox, angle

In [548]:
def shiftPair_Rt(rtPair, rtList, kernel_bandwidth):
    totalW = 0.0
    shifted_pair = [rtPair[0], rtPair[1], rtPair[2], rtPair[3], [np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0])]]
    
    for i in range(len(rtList)):
        distanceMetric = distanceMetricRt(rtPair, rtList[i])
        
        if distanceMetric[0] < kernel_bandwidth[0] and distanceMetric[1] < kernel_bandwidth[1]:            
            # A list cannot directly handle a mathematical operations, while array can
            weight = epanechnikov_weight(distanceMetric, kernel_bandwidth)
            shifted_pair[4][0] += rtList[i][4][0] * weight
            shifted_pair[4][1] += rtList[i][4][1] * weight
            totalW += weight
    
    if totalW > 0:
        shifted_pair[4][0] = shifted_pair[4][0] / np.linalg.norm(shifted_pair[4])
        shifted_pair[4][1] = shifted_pair[4][1] / totalW
        
        return shifted_pair
    else:
        return rtPair

In [554]:
#lxcPairs
def meanShift_Rt(lrPairs, kernel_bandwidth):
    stop_moving = [False for i in range(len(lrPairs))]
    shiftedPairs = lrPairs.copy()
        
    while True:
        max_shift_distance = 0
        
        for i in range(len(lrPairs)):
            if not stop_moving[i]:
                pair_new = shiftPair_Rt(shiftedPairs[i], lrPairs, kernel_bandwidth)  
                
                distance_metric = distanceMetricRt(pair_new, shiftedPairs[i])
                shift_distance = distance_metric[0] * distance_metric[1]
                max_shift_distance = max(max_shift_distance, shift_distance)
                shiftedPairs[i] = pair_new
                
                if shift_distance < EPSILON:
                    stop_moving[i] = True
                
        if max_shift_distance < EPSILON:
            break
            
    return shiftedPairs

In [550]:
def clusterRt(pairs, shiftedPairs):
    cluster_set = []
    
    for i in range(0, len(shiftedPairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRt(shiftedPairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points, shifted points
            cluster = [shiftedPairs[i], [], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
        cluster_set[c][3].append(shiftedPairs[i])
    
    return cluster_set

In [563]:
shiftedPairsRt = meanShift_Rt(lcrPairs, KERNEL_BW)

KeyboardInterrupt: 

In [ ]:
#Usar Kd-Tree o prescindir del mean shift.

In [531]:
n_clusterRt = clusterRt(lrPairs, shiftedPairsRt)

In [532]:
len(n_clusterRt)

128

In [564]:
def clusterRt2(pairs):
    cluster_set = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRt(pairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [pairs[i], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
    
    return cluster_set

In [567]:
n_cluster2_Rt = clusterRt2(lcrPairs)

In [571]:
for x in range(0, len(n_cluster2_Rt)):
    if len(n_cluster2_Rt[x][1]) > 20:
        print(x, len(n_cluster2_Rt[x][1]))

1 42
4 2015
7 34
8 95
9 67
10 68
11 86
13 60
16 98
18 30
22 23
25 29
28 54
30 46
31 27
32 100
33 37
34 59
38 121
39 23
40 100
42 116
45 28
52 33
54 40
62 41
87 28
92 21
95 30
106 68
139 55
202 47
210 21
231 34


In [574]:
n_cluster2_Rt[4][0], len(n_cluster2_Rt[4][1])

([259,
  260,
  259,
  273,
  (array([-0.,  0.,  1.]),
   array([6.07508630e-06, 3.37345461e-06, 0.00000000e+00]))],
 2015)

In [348]:
distanceMetricRx(shiftedPairsFx[6], n_cluster[0][0])

(0.41745188130705396, 1.2311834058802362)

In [142]:
#x1 = [127, 159, np.array([ 0.8375182 , -0.32672394,  0.43796659]), -2.4115509526708188]
#x2 = [127, 159, np.array([ 0.8375182 , -0.32672394,  0.43796659]), -2.4115509526708188]

x1 = np.array([ 0.8375182 , -0.32672394,  0.43796659])
x2 = np.array([ 0.8375182 , -0.32672394,  0.43796659])
#math.acos(abs(x1[2].dot(x2[2])))
abs(x1.dot(x2)) #math.acos(abs(x1.dot(x2)))

1.0000000022565916

In [130]:
#def DetectionReflectional(baseRx):
shifted_point = [lxcPairs[2][0],  lxcPairs[2][1], np.array([0.0, 0.0, 0.0]), 0]
lista = lxcPairs[2]
shifted_point[2] = lista[2]
shifted_point[3] = lista[3]
shifted_point

In [266]:
distanceMetricRx(lxcPairs[22], lxcPairs[250])[0], distanceMetricRx(lxcPairs[271], lxcPairs[501])[1]

(0.003121760987498162, 0.0)

In [575]:
lDist = []
similar_pairs = []

for i in range(len(lxcPairs)):
    for j in range(i + 1, len(lxcPairs)):
        #lDist.append(distanceMetricRx(lxcPairs[i], lxcPairs[j]))
        origin = np.array([0, 0, 0])
        projectionA = projectPointPlane(origin, lxcPairs[i][2], lxcPairs[i][3])
        projectionB = projectPointPlane(origin, lxcPairs[j][2], lxcPairs[j][3])
        distance = np.linalg.norm(projectionA - projectionB) / diagonal_bbox
        if abs(np.dot(lxcPairs[i][2], lxcPairs[j][2])) > 0.99 and distance < 0.05: #0.98
            similar_pairs.append([i, j, distance, math.acos(min(abs(np.dot(lxcPairs[i][2], lxcPairs[j][2])), 1.0))])
            #print(i, j, distance, math.acos(min(abs(np.dot(lxcPairs[i][2], lxcPairs[j][2])), 1.0)))
            
len(similar_pairs)        
#np.percentile(lDist, 10), np.percentile(lDist, 25), np.percentile(lDist, 40), np.percentile(lDist, 55), np.percentile(lDist, 70), np.percentile(lDist, 85)

5389

In [194]:
k1, k2

(array([ 0.08726476,  0.0731308 ,  0.0400593 , ..., -2.54055871,
        -2.54062088, -2.54056058]),
 array([5.67214749e+00, 6.04994646e+00, 5.55570293e+00, ...,
        1.41687329e-03, 1.41637564e-03, 1.07589602e-04]))

In [281]:
lxcPairs[22], lxcPairs[250], lxcPairs[271], lxcPairs[501]

([259,
  285,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([0., 0., 0.])],
 [550,
  570,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([-0.02056, -0.02056,  0.     ])],
 [587,
  597,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([0., 0., 0.])],
 [996,
  1020,
  array([ 0.70710678,  0.70710678, -0.        ]),
  array([-0.03701, -0.03701,  0.     ])])

In [208]:
v[lxcPairs[22][0]], v[lxcPairs[22][1]], v[lxcPairs[271][0]], v[lxcPairs[271][1]]

(array([ 0.19729, -0.99185,  6.80349]),
 array([ 0.99185, -0.19729,  6.80349]),
 array([-1.25006, -0.24865,  5.08451]),
 array([0.24865, 1.25006, 5.08451]))

In [299]:
x = np.array([7, 8])
y = np.array([6, 10])

c = x > y
print(any(c), all(c))

True False


In [301]:
cx = distanceMetricRx(lxcPairs[22], lxcPairs[250]) > y
print(any(c), all(c))

True False


In [305]:
p3 = []
for i in range(0, len(p3)):
    print(i)

In [422]:
set_clus = []
clus = [[0.0, 2.0, 4.0], [], []]
set_clus.append(clus)
set_clus, clus[1], clus[2]

([[[0.0, 2.0, 4.0], [], []]], [], [])

In [315]:
clus2 = [[0.0, 2.0, 4.0], np.array([]), np.array([])]

In [320]:
clus[1].append(lxcPairs[0])
clus[2].append(shiftedPairsFx[0])

In [321]:
clus

[[0.0, 2.0, 4.0],
 [[4,
   33,
   array([ 0.14220045, -0.65268028,  0.74416899]),
   array([ 0.56531429, -2.59471394,  2.95842496])]],
 [[4,
   33,
   array([ 0.14220045, -0.65268028,  0.74416899]),
   array([ 0.56531429, -2.59471394,  2.95842496])]]]

In [ ]:
k1, k2